In [ ]:
#Do not chance these dependencies
try:
    import pytest
except: !pip install pytest
    import pytest
# Import your dependencies here
try: from bs4 import BeautifulSoup
except: !pip install bs4
    from bs4 import BeautifulSoup

try: import spacy
except: !pip install spacy
    import spacy
import requests, json

*I made a function that scrapes the BBC News homepage and takes any links that are tageted at another /news page.*

In [ ]:
# gets all links from bbc news home page if they link to another /news story. 
def get_all_links():
    full_url = 'https://www.bbc.co.uk/news'
    get_links = requests.get(full_url).text
    full_soup = BeautifulSoup(get_links, 'html.parser')

    urls = []
    for a in full_soup.find_all('a', class_ = 'gs-c-promo-heading', href=True):
        if '/news' in a['href'] and 'https://' not in a['href']:
            urls.append('https://www.bbc.co.uk' + a['href'])
    urls = list(dict.fromkeys(urls))
    
    for url in urls:
        article = requests.get(url).text
        soup = BeautifulSoup(article, 'html.parser')
        title = soup.title.text
        print(urls.index(url), title, url)        
    return urls, full_soup

*The links are output with enumeration so you may choose if there's a particular story you would like to run through the scraper.*

In [ ]:
# prints with leading list index so you can use that if you want to query just one in the full scrape.
urls, full_soup = get_all_links()

*Then, you may choose to supply the BBC scraper with one URL or many from the list created by the homepage scraper.*

In [ ]:
def bbc_scraper(url):
    article = requests.get(url).text
    soup = BeautifulSoup(article, 'html.parser')
    content = soup.find(property='articleBody')
    try: # simple error handling. It will fail if link leads to page which is not a text article as the searched for tags will not exist, eg video
        content = [content.text for content in content.find_all('p')]
        content = ' '.join(content)
        title = soup.find(class_='story-body__h1').text
        date = soup.find('ul', class_='mini-info-list').text.strip()


    except:
        title = soup.title.text
        content = ' '
        date = ' '
        print('Invalid webpage for text analysis: ' + title + ' ' + url)
        pass
    
    results_json = {'URL':url, 'Title':title, 'Date_published':date, 'Content':content}
    results_json = json.dumps(results_json)
    return results_json

*If any of the URLs in the list created from scraping the homepage are not valid text based news stories then you will get a warning.*

In [ ]:
# to present all json objects for each url
for url in urls:
    print(bbc_scraper(url))

*The following three cells allow you to choose you query method*

In [ ]:
# This will pull the content element for any story in the list generated, sliced on the enumerated value.
string = json.loads(bbc_scraper(urls[5]))['Content']

In [ ]:
# This will allow you to query a URL not in the list, so say one that is no longer on the News homepage
url = 'https://www.bbc.co.uk/news/election-us-2020-53876958'
string = json.loads(bbc_scraper(url))['Content']

In [ ]:
# To take all of the outputed JSON object rather than just the content element
url = 'https://www.bbc.co.uk/news/election-us-2020-53876958'
string = bbc_scraper(url)

*Import the Space module required for entity extraction*

In [ ]:
# import spacy language model
nlp = spacy.load('en_core_web_sm')

*Function to extract entities from the string created above, by which ever method you chose.*

In [ ]:
def extract_entities(string):
    people = []
    places = []
    organisations = []
    doc = nlp(string)
    for token in doc.ents:
        if token.label_ == 'PERSON':
            people.append(str(token)) #all added as strings to enable easier manipulation later
        if token.label_ == 'GPE' or token.label_ == 'FAC': 
            places.append(str(token))
        if token.label_ == 'ORG':
            organisations.append(str(token))

    # deduping the lists to remove repeatedly mentioned entities in the article
    people = list(dict.fromkeys(people))
    places = list(dict.fromkeys(places))
    organisations = list(dict.fromkeys(organisations))

    # create the json output
    ent_out = {'people':people,'places':places,'organisations':organisations}
    entities_json = json.dumps(ent_out)
    #entities_json = json.loads(ent_out)
    return entities_json

In [ ]:
json.loads(extract_entities(string))


In [ ]:
####################################################################
# Test cases

def test_bbc_scrape():
    results = {'URL': 'https://www.bbc.co.uk/news/uk-52255054',
                'Title': 'Coronavirus: \'We need Easter as much as ever,\' says the Queen',
                'Date_published': '11 April 2020',
                'Content': '"Coronavirus will not overcome us," the Queen has said, in an Easter message to the nation. While celebrations would be different for many this year, she said: "We need Easter as much as ever." Referencing the tradition of lighting candles to mark the occasion, she said: "As dark as death can be - particularly for those suffering with grief - light and life are greater." It comes as the number of coronavirus deaths in UK hospitals reached 9,875. Speaking from Windsor Castle, the Queen said many religions had festivals celebrating light overcoming darkness, which often featured the lighting of candles. She said: "They seem to speak to every culture, and appeal to people of all faiths, and of none. "They are lit on birthday cakes and to mark family anniversaries, when we gather happily around a source of light. It unites us." The monarch, who is head of the Church of England, said: "As darkness falls on the Saturday before Easter Day, many Christians would normally light candles together.  "In church, one light would pass to another, spreading slowly and then more rapidly as more candles are lit. It\'s a way of showing how the good news of Christ\'s resurrection has been passed on from the first Easter by every generation until now." As far as we know, this is the first time the Queen has released an Easter message. And coming as it does less than a week since the televised broadcast to the nation, it underlines the gravity of the situation as it is regarded by the monarch. It serves two purposes really; it is underlining the government\'s public safety message, acknowledging Easter will be difficult for us but by keeping apart we keep others safe, and the broader Christian message of hope and reassurance.  We know how important her Christian faith is, and coming on the eve of Easter Sunday, it is clearly a significant time for people of all faiths, but particularly Christian faith. She said the discovery of the risen Christ on the first Easter Day gave his followers new hope and fresh purpose, adding that we could all take heart from this.  Wishing everyone of all faiths and denominations a blessed Easter, she said: "May the living flame of the Easter hope be a steady guide as we face the future." The Queen, 93, recorded the audio message in the White Drawing Room at Windsor Castle, with one sound engineer in the next room.  The Palace described it as "Her Majesty\'s contribution to those who are celebrating Easter privately".  It follows a speech on Sunday, in which the monarch delivered a rallying message to the nation. In it, she said the UK "will succeed" in its fight against the coronavirus pandemic, thanked people for following government rules about staying at home and praised those "coming together to help others". She also thanked key workers, saying "every hour" of work "brings us closer to a return to more normal times".'}
    scraper_result = bbc_scraper('https://www.bbc.co.uk/news/uk-52255054')
    assert json.loads(scraper_result) == results


def test_extract_entities_amazon_org():
    input_string = "I work for Amazon."
    results_dict = {'people':[],
                    'places':[],
                    'organisations': ['Amazon']
                    }
    extracted_entities_results = extract_entities(input_string)
    assert json.loads(extracted_entities_results) == results_dict


def test_extract_entities_name():
    input_string = "My name is Bob"
    results_dict = {'people':['Bob'],
                    'places':[],
                    'organisations': []
                    }
    extracted_entities_results = extract_entities(input_string)
    assert json.loads(extracted_entities_results) == results_dict

In [ ]:
test_bbc_scrape()
test_extract_entities_amazon_org()
test_extract_entities_name()